In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder

In [4]:
#loading master_data2.csv
data2 = pd.read_csv('saved_datasets/master_data2.csv')

In [5]:
#total unique user_id
data2['user_id'].nunique()

2192

In [6]:
#reordered count/total rows : reordered = 1 percentage
data2[data2['reordered'] == 1].shape[0]/data2.shape[0]

0.5835062884714314

In [7]:
# Spliting the data train validation and test using stratified sampling on reordered
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

# Spliting the data train validation and test using stratified sampling on reordered
train, test = train_test_split(data2, test_size=0.2, stratify=data2['reordered'], random_state=42)
train, val = train_test_split(train, test_size=0.2, stratify=train['reordered'], random_state=42)

train.shape, val.shape, test.shape


((216472, 15), (54118, 15), (67648, 15))

In [8]:
# percentage of reordered = 1 in train, val and test
train[train['reordered'] == 1].shape[0]/train.shape[0], val[val['reordered'] == 1].shape[0]/val.shape[0], test[test['reordered'] == 1].shape[0]/test.shape[0]

(0.5835073358217229, 0.5835027162866329, 0.5835057947019867)

In [9]:
train.head()

,order_id,product_id,add_to_cart_order,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department,reordered
189439,1974607,32611,12,2063,prior,3,0,15,21.0,Honey Almond Butter Single,88,13,spreads,pantry,0
218925,2291360,37382,28,184,prior,11,0,11,4.0,Organic Low Fat Minestrone Soup,69,15,soup broth bouillon,canned goods,0
188234,1961788,34005,3,912,prior,2,4,9,6.0,Black Seedless Grapes,24,4,fresh fruits,produce,0
35268,361487,41844,10,1990,prior,31,3,11,5.0,Honey Nut Cheerios,121,14,cereal,breakfast,1
138856,1449276,30427,6,236,prior,42,3,15,18.0,Accents Organic Sprouted Quinoa Trio,63,9,grains rice dried goods,dry goods pasta,0


In [10]:
#function to drop order_id, eval_set, product_id, aisle_id and department_id
def drop_columns(data):
    data = data.drop(['order_id', 'eval_set', 'product_name', 'aisle_id', 'department_id'], axis=1)
    return data


#adding a feature: average days_since_prior_order for each user_id

def avg_days_since_prior_order(data):
    days_since_prior_order = data.groupby('user_id')['days_since_prior_order'].mean().reset_index()
    data = data.merge(days_since_prior_order, on='user_id', how='left')
    data = data.rename(columns={'days_since_prior_order_x':'days_since_prior_order', 'days_since_prior_order_y':'avg_days_since_prior_order'})
    return data


#most_common order_dow for each user_id : mode of order_dow for each user_id
def most_common_order_dow(data):
    order_dow = data.groupby('user_id')['order_dow'].agg(lambda x:x.value_counts().index[0]).reset_index()
    data = data.merge(order_dow, on='user_id', how='left')
    data = data.rename(columns={'order_dow_x':'order_dow', 'order_dow_y':'most_common_order_dow'})
    return data


#spliting order_hour_of_day into 4 categories
def split_order_hour_of_day(data):
    data['order_hour_of_day'] = pd.cut(data['order_hour_of_day'], bins=[0, 6, 12, 18, 24], labels=['night', 'morning', 'afternoon', 'evening'])
    return data


#total number of orders for each user_id
def total_orders(data):
    total_orders = data.groupby('user_id')['order_number'].max().reset_index()
    data = data.merge(total_orders, on='user_id', how='left')
    data = data.rename(columns={'order_number_x':'order_number', 'order_number_y':'total_orders'})
    return data

#average number of products in each order for each user_id
def avg_products(data):
    avg_products = data.groupby('user_id')['add_to_cart_order'].mean().reset_index()
    data = data.merge(avg_products, on='user_id', how='left')
    data = data.rename(columns={'add_to_cart_order_x':'add_to_cart_order', 'add_to_cart_order_y':'avg_products'})
    return data

#fill na values in days_since_prior_order with 0
def fill_na(data):
    data['days_since_prior_order'] = data['days_since_prior_order'].fillna(0)
    return data

##imputing order_hour_of_day with most common order_hour_of_day
def impute_order_hour_of_day(data):
    data['order_hour_of_day'] = data['order_hour_of_day'].fillna(data['order_hour_of_day'].mode()[0])
    return data



#function to seperate features and target
def feature_target(data):
    X = data.drop('reordered', axis=1)
    y = data['reordered']
    return X, y

#drop all null1 rows
def drop_null_rows(data):
    data = data.dropna()
    return data

# #function to encode product name
# def encode_product_name(data):
#     oe = OrdinalEncoder()
#     data['product_name'] = oe.fit_transform(data['product_name'].values.reshape(-1, 1))
#     return data, oe


#one hot encoding for other categorical columns
def one_hot_encoding(data):
    data = pd.get_dummies(data, columns=['order_dow', 'order_hour_of_day', 'aisle', 'department', 'most_common_order_dow'])
    return data



In [11]:
#dropping columns
train = drop_columns(train)
train = avg_days_since_prior_order(train)
train = most_common_order_dow(train)
train = split_order_hour_of_day(train)
train = total_orders(train)
train = avg_products(train)
train = fill_na(train)
train = impute_order_hour_of_day(train)
train = drop_null_rows(train)
train = one_hot_encoding(train)
# train, oe = encode_product_name(train)


train.head()

,product_id,add_to_cart_order,user_id,order_number,days_since_prior_order,reordered,avg_days_since_prior_order,total_orders,avg_products,order_dow_0,...,department_pets,department_produce,department_snacks,most_common_order_dow_0,most_common_order_dow_1,most_common_order_dow_2,most_common_order_dow_3,most_common_order_dow_4,most_common_order_dow_5,most_common_order_dow_6
0,32611,12,2063,3,21.0,0,20.782609,4,6.387097,1,...,0,0,0,0,0,0,0,0,0,1
1,37382,28,184,11,4.0,0,21.355769,12,9.649573,1,...,0,0,0,1,0,0,0,0,0,0
2,34005,3,912,2,6.0,0,7.365854,7,6.180000,0,...,0,1,0,0,1,0,0,0,0,0
3,41844,10,1990,31,5.0,1,9.385915,42,8.796089,0,...,0,0,0,0,1,0,0,0,0,0
4,30427,6,236,42,18.0,0,6.155280,45,8.036697,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
#function to encode product name in test and val

def encode_product_name_val_test(data, oe):
    data['product_name'] = oe.transform(data['product_name'].values.reshape(-1, 1))
    return data


In [13]:
#preprocessing val 
val = drop_columns(val)
val = avg_days_since_prior_order(val)
val = most_common_order_dow(val)
val = split_order_hour_of_day(val)
val = total_orders(val)
val = avg_products(val)
val = fill_na(val)
val = impute_order_hour_of_day(val)
val = drop_null_rows(val)
val = one_hot_encoding(val)
# val = encode_product_name_val_test(val, oe)

val.head()


,product_id,add_to_cart_order,user_id,order_number,days_since_prior_order,reordered,avg_days_since_prior_order,total_orders,avg_products,order_dow_0,...,department_pets,department_produce,department_snacks,most_common_order_dow_0,most_common_order_dow_1,most_common_order_dow_2,most_common_order_dow_3,most_common_order_dow_4,most_common_order_dow_5,most_common_order_dow_6
0,7649,9,210,63,3.0,1,3.221311,100,8.868852,0,...,0,0,0,1,0,0,0,0,0,0
1,33731,14,462,54,7.0,1,5.718750,59,10.696970,0,...,0,0,0,0,1,0,0,0,0,0
2,43086,4,465,12,1.0,0,9.378378,41,8.756757,0,...,0,1,0,0,0,0,0,0,1,0
3,28204,3,1866,18,4.0,1,4.044944,97,4.648352,0,...,0,1,0,0,0,0,0,0,1,0
4,13263,7,1072,15,6.0,1,7.100000,15,8.050000,0,...,0,0,0,0,1,0,0,0,0,0


In [14]:
#preprocessing test
test = drop_columns(test)
test = avg_days_since_prior_order(test)
test = most_common_order_dow(test)
test = split_order_hour_of_day(test)
test = total_orders(test)
test = avg_products(test)
test = fill_na(test)
test = impute_order_hour_of_day(test)
test = drop_null_rows(test)
test = one_hot_encoding(test)
# test = encode_product_name_val_test(test, oe)

test.head()

,product_id,add_to_cart_order,user_id,order_number,days_since_prior_order,reordered,avg_days_since_prior_order,total_orders,avg_products,order_dow_0,...,department_pets,department_produce,department_snacks,most_common_order_dow_0,most_common_order_dow_1,most_common_order_dow_2,most_common_order_dow_3,most_common_order_dow_4,most_common_order_dow_5,most_common_order_dow_6
0,5258,12,1562,2,10.0,0,7.950000,9,9.622222,0,...,0,0,0,0,0,1,0,0,0,0
1,22900,2,1604,4,30.0,0,30.000000,4,2.500000,0,...,0,0,0,0,0,0,0,1,0,0
2,41570,3,348,20,2.0,0,12.086957,29,2.708333,0,...,0,0,0,0,0,0,0,0,1,0
3,38662,14,830,4,21.0,1,28.000000,6,9.000000,1,...,0,0,0,1,0,0,0,0,0,0
4,47176,1,1546,19,7.0,1,10.229167,49,5.448980,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
X_train, y_train = feature_target(train)
X_val, y_val = feature_target(val)
X_test, y_test = feature_target(test)

In [16]:
#save the preprocessed data
X_train.to_csv('saved_datasets/X_train.csv', index=False)
y_train.to_csv('saved_datasets/y_train.csv', index=False)
X_val.to_csv('saved_datasets/X_val.csv', index=False)
y_val.to_csv('saved_datasets/y_val.csv', index=False)
X_test.to_csv('saved_datasets/X_test.csv', index=False)
y_test.to_csv('saved_datasets/y_test.csv', index=False)